## PUBG Modelling

플레이어들이 전장에서 벌이는 모든 행위를 분류하여 승리를 예측하기 위한 프로젝트

## Column 별 정보
- ASSISTS : 자신에 의해 사살되지는 못했지만 자신의 데미지 지분율이 가장 높은 적의 수
- BOOSTS : 도핑 아이템 사용 횟수(에너지 드링크, 진통제, 아드레날린 주사기)
- DAMAGEDEALT : 적에게 가한 전체 데미지
- DBNOS : 기절시킨 적의 수
- GROUPID : 경기 내 그룹 식별 ID
- HEADSHOTKILLS : 헤드샷으로 죽인 적의 수
- HEALS : 회복 아이템 사용 횟수(붕대, 구급상자, 의료용 키트)
- ID : 해당 데이터 ID
- KILLPLACE : 경기 내 적을 죽인 적 수의 순위
- KILLPOINTS : 유저의 ELO레이팅(죽인 적 수 기반)
- KILLS : 경기 내 적을 죽인 수
- KILLSTREAKS : 짧은 시간 내에 연속으로 적을 죽인 수
- LONGESTKILL : 사살한 적까지의 거리의 최대값
- MATCHDURATION : 경기가 진행된 시간(단위:초)
- MATCHID : 경기 식별 ID
- MATCHTYPE : 경기 모드( ex: 솔로, 듀오, 스쿼드 )
- MAXPLACE : 경기 내 총 인원 수
- NUMGROUP : 경기 내 실제 참여 인원 수
- RANKPOINTS : 유저의 ELO 레이팅
- REVIVES : 유저가 팀원을 부활시킨 횟수
- RIDEDISTANCE : 이동수단을 통해 이동한 거리(단위:m)
- ROADKILLS : 이동수단으로 살해한 적의 수
- SWIMDISTANCE : 수영으로 이동한 거리(단위:m)
- TEAMKILLS : 같은 팀원을 살해한 수
- VEHICLEDESTROYS : 파괴한 이동수단의 수
- WALKDISTANCE : 도보로 이동한 거리(단위:m)
- WEAPONSACQUIRED : 획득한 무기의 수
- WINPOINTS : 유저의 ELO 레이팅(승리 횟수 기반)
- WINPLACEPERC : 현재 경기에서의 백분위 기반 유저의 순위(종속변수)

## 라이브러리 및 데이터 불러오기

In [ ]:
# 데이터 분석
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# 학습을 위한 라이브러리 세팅
from sklearn.linear_model import LinearRegression   
from sklearn.linear_model import Lasso              
from sklearn.linear_model import Ridge             
from xgboost.sklearn import XGBRegressor            
from lightgbm.sklearn import LGBMRegressor 
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from lightgbm.sklearn import LGBMClassifier        
from sklearn.metrics import mean_absolute_error
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from functools import partial

# VIF
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Validation
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

# Hyper parameter tuning
import optuna
import sklearn.metrics as metrics

# others 
import random
import scipy as sp
from scipy import stats
import sys
import gc
import os
print(os.listdir("./data/pubg-finish-placement-prediction/"))
import warnings                      
warnings.filterwarnings("ignore")

In [48]:
train = pd.read_csv("./data/pubg-finish-placement-prediction/train_V2.csv")
test = pd.read_csv("./data/pubg-finish-placement-prediction/test_V2.csv")
submission = pd.read_csv("./data/pubg-finish-placement-prediction/sample_submission_V2.csv")

## 함수

In [3]:
# Memory saving function credit to https://www.kaggle.com/gemartin/load-data-reduce-memory-usage
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
#     start_mem = df.memory_usage().sum() / 1024**2
#     print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

#     end_mem = df.memory_usage().sum() / 1024**2
#     print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
#     print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

In [4]:
def sorted_corr(data, column) :
    df_corr = data.corr()
    df_corr = df_corr.apply(lambda x: round(x ,2))
    df_corr = df_corr.unstack()
    df_corr = pd.DataFrame(df_corr[column][df_corr[column]<1].sort_values(ascending=False), columns=['Correlation'])
    df_corr = df_corr.style.background_gradient(cmap='coolwarm_r')
    return df_corr

In [5]:
def toVIF(features): 
    return pd.DataFrame({
        "feature": features.columns,
        "VIF": [variance_inflation_factor(features.values, idx)
                for idx in range(features.shape[1])]
    })

In [6]:
def OLS_summary(features, target):
    sm_feature = sm.add_constant(features)
    model = sm.OLS(target, sm_feature).fit()
    print(model.summary2())

In [7]:
from sklearn.linear_model import LinearRegression   # 1. Linear Regression
from sklearn.linear_model import Lasso              # 2. Lasso
from sklearn.linear_model import Ridge              # 3. Ridge
from xgboost.sklearn import XGBRegressor            # 4. XGBoost
from lightgbm.sklearn import LGBMRegressor          # 5. LightGBM
from sklearn.metrics import mean_absolute_error


def Linear_training_mae(target, features):
    model = LinearRegression().fit(features,target)
    pred = model.predict(features)
    mae = np.round(mean_absolute_error(target, pred), 4)
    return mae
def Lasso_training_mae(target, features):
    model = Lasso().fit(features,target)
    pred = model.predict(features)
    mae = np.round(mean_absolute_error(target, pred), 4)
    return mae
def Ridge_training_mae(target, features):
    model = Ridge().fit(features,target)
    pred = model.predict(features)
    mae = np.round(mean_absolute_error(target, pred), 4)
    return mae
def XGB_training_mae(target, features):
    model = XGBRegressor().fit(features,target)
    pred = model.predict(features)
    mae = np.round(mean_absolute_error(target, pred), 4)
    return mae
def LGBM_training_mae(target, features):
    model = LGBMRegressor().fit(features,target)
    pred = model.predict(features)
    mae = np.round(mean_absolute_error(target, pred), 4)
    return mae
def trainings_mae(target, features):
    print("1. Linear Regression\t, train=%.4f" % Linear_training_mae(target, features))
    print("2. Lasso\t\t, train=%.4f" % Lasso_training_mae(target, features))        
    print("3. Ridge\t\t, train=%.4f" % Ridge_training_mae(target, features))
    print("4. XGBoost\t\t, train=%.4f" % XGB_training_mae(target, features))
    print("5. LightGBM\t\t, train=%.4f" % LGBM_training_mae(target, features))

In [8]:
def one_hot_encoding(data):
    train_OHE = pd.get_dummies(data, columns=["matchType"])
    train_OHE = reduce_mem_usage(train_OHE)
    return train_OHE

In [9]:
def ordinal_encoding(data):
    train_OE = data.copy()
    train_OE['matchType'] = train_OE['matchType'].map({
        'solo':1,
        'solo-fpp':2,
        'duo':3,
        'duo-fpp':4,
        'squad':5,
        'squad-fpp':6,
        'normal-duo':7,
        'normal-duo-fpp':8,
        'normal-solo':9,
        'normal-solo-fpp':10,
        'normal-squad':11,
        'normal-squad-fpp':12,
        'crashfpp':13,
        'crashtpp':14,
        'flarefpp':15,
        'flaretpp':16
        })
    train_OE = reduce_mem_usage(train_OE)
    return train_OE

In [10]:
# optuna RandomForest
def optimizer_RF(trial, X, y, K):
    n_estimators = trial.suggest_int("n_estimators", 50, 200)
    max_depth = trial.suggest_int("max_depth", 8, 30)
    max_features = trial.suggest_categorical("max_features", ['auto', 'sqrt', 'log2'])
    evaluation_metric = mean_absolute_error
    
    model = RandomForestRegressor(n_estimators=n_estimators,
                                  max_depth=max_depth,
                                  max_features=max_features,
                                  n_jobs=-1,
                                  random_state=0xC0FFEE)
    
    folds = KFold(n_splits=K)
    scores = []
    
    for train_idx, val_idx in folds.split(X, y):
        X_train = X.iloc[train_idx, :]
        y_train = y.iloc[train_idx]
        
        X_val = X.iloc[val_idx, :]
        y_val = y.iloc[val_idx]
        
        model.fit(X_train, y_train)
        preds = model.predict(X_val)
        score = evaluation_metric(y_val, preds)
        scores.append(score)
        
    return np.mean(scores)

In [11]:
# Optuna LightGBM
def optimizer_LGBM(trial, X, y, K):
    import os
    param = {
        'objective': 'regression', # 회귀
        'verbose': 0,
        'max_depth': trial.suggest_int('max_depth', 8, 20),
        'learning_rate': trial.suggest_loguniform("learning_rate", 1e-8, 1e-2),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'subsample': trial.suggest_loguniform('subsample', 0.4, 1)
        #"device" : 'gpu'
    }

    model = LGBMRegressor(**param, n_jobs=os.cpu_count())
    evaluation_metric = mean_absolute_error
    
    folds = KFold(n_splits=K)
    scores = []
    
    for train_idx, val_idx in folds.split(X, y):
        
        # X_train,X_val = X[train_idx],X[val_idx]
        # y_train,y_val = y[train_idx],y[val_idx]
        
        X_train = X.iloc[train_idx, :]
        y_train = y.iloc[train_idx]
        
        X_val = X.iloc[val_idx, :]
        y_val = y.iloc[val_idx]
        
        model.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=25)
        preds = model.predict(X_val)
        score = evaluation_metric(y_val, preds)
        scores.append(score)
    
    return np.mean(scores)

In [45]:
def dropOutlier (df):
    def dropIdx(df, idx) :
        df.drop(index=idx, inplace=True)
        dropIdx.dpIdx_sum +=len(idx)
        return df
    dropIdx.dpIdx_sum = 0

    print("Pre-Processing...")
    for i in df.columns.to_list() :
        df.drop(index=df[df[i].isnull()==True].index, inplace=True)
        
    print("Droping Outliers...")

    vip_features = ["assists","boosts","DBNOs","heals","kills","killStreaks","walkDistance", "revives", "roadKills", "vehicleDestroys"]

    group = df.groupby('groupId').count()
    df = dropIdx(df, df[df.groupId.isin(group[group["Id"]>group["Id"].quantile(0.9999)].index)==True].index) #수치고려 가능

    for col in (vip_features + ["damageDealt","longestKill", "rideDistance", "swimDistance","weaponsAcquired", "matchDuration"]):
        df = dropIdx(df, df[df[col]>df[col].quantile(0.999)].index)
    
    for col in vip_features:
        df = dropIdx(df, df[df["walkDistance"]<df[col]].index)

    df = dropIdx(df, df[df.groupby('matchId')['kills'].transform('max')  > df.groupby('matchId')['Id'].transform('count')  ].index)
    df = dropIdx(df, df[(df['rideDistance']==0) & (df['roadKills']>0)  ].index)

    #edge case
    df.loc[(df.maxPlace>1)&(df.numGroups==1), "maxPlace"] = 1

    print(f"{dropIdx.dpIdx_sum} Columns has deleted!") 

    del vip_features, group      
    gc.collect()
    
    return df

In [36]:
def feature_enginnering(df):
    # create columns
    print("Making columns...")
    stat_feature = ["assists","boosts","DBNOs","heals","kills","killStreaks","walkDistance", "revives", "roadKills", "vehicleDestroys","damageDealt","longestKill", "rideDistance", "swimDistance","weaponsAcquired"]
    stat_list = ["max","mean","median","min"]
    for col in stat_feature :
        for stat in stat_list:
            df[f"{col}_{stat}"] = df.groupby("groupId")[col].transform(stat)
            df[f"{col}_{stat}_Place"] = df.groupby("matchId")[f"{col}_{stat}"].transform('rank', ascending=False)
    print(len(stat_feature)*len(stat_list)+1, f"columns Made! now {len(df.columns)}column in DF.")
    
    # create kiilRank using matchId, kills 
    df["killRank"] = df.groupby("matchId")["kills"].rank("dense", ascending=False)
    
    # drop unnecessary columns 
    print('Dropping columns...')
    df = df.drop(["Id", "groupId","matchId"], axis=1)
    df = ordinal_encoding(df)
    df = df.drop(['killPlace'], axis=1)
    df = df.drop(['damageDealt'], axis=1)
    df = df.drop(['numGroups'], axis=1)
    df = df.drop(['killPoints','rankPoints','winPoints'], axis=1)
    
    print(len(df.columns), "columns in df")    
    reduce_mem_usage(df)
    del stat_feature
    gc.collect()

    return df

In [ ]:
def get_Output(df):
    print("Dropping Outlers...")
    df = dropOutlier(df)
    print("\n" + "-"*30 )
    print("Doing feature engineering...")
    df = feature_enginnering(df)
    return df 

In [ ]:
def sampling(df,n):
    idx = sorted(np.random.permutation(len(df))[:n])
    return df.iloc[idx].copy()

## 전처리

In [13]:
train = reduce_mem_usage(train)
test = reduce_mem_usage(test)

In [14]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4446966 entries, 0 to 4446965
Data columns (total 29 columns):
 #   Column           Dtype  
---  ------           -----  
 0   Id               object 
 1   groupId          object 
 2   matchId          object 
 3   assists          int8   
 4   boosts           int8   
 5   damageDealt      float16
 6   DBNOs            int8   
 7   headshotKills    int8   
 8   heals            int8   
 9   killPlace        int8   
 10  killPoints       int16  
 11  kills            int8   
 12  killStreaks      int8   
 13  longestKill      float16
 14  matchDuration    int16  
 15  matchType        object 
 16  maxPlace         int8   
 17  numGroups        int8   
 18  rankPoints       int16  
 19  revives          int8   
 20  rideDistance     float16
 21  roadKills        int8   
 22  swimDistance     float16
 23  teamKills        int8   
 24  vehicleDestroys  int8   
 25  walkDistance     float16
 26  weaponsAcquired  int16  
 27  winPoints   

In [15]:
train.isna().sum()

Id                 0
groupId            0
matchId            0
assists            0
boosts             0
damageDealt        0
DBNOs              0
headshotKills      0
heals              0
killPlace          0
killPoints         0
kills              0
killStreaks        0
longestKill        0
matchDuration      0
matchType          0
maxPlace           0
numGroups          0
rankPoints         0
revives            0
rideDistance       0
roadKills          0
swimDistance       0
teamKills          0
vehicleDestroys    0
walkDistance       0
weaponsAcquired    0
winPoints          0
winPlacePerc       1
dtype: int64

In [16]:
train = train.dropna(axis=0)

In [17]:
train.isna().sum()

Id                 0
groupId            0
matchId            0
assists            0
boosts             0
damageDealt        0
DBNOs              0
headshotKills      0
heals              0
killPlace          0
killPoints         0
kills              0
killStreaks        0
longestKill        0
matchDuration      0
matchType          0
maxPlace           0
numGroups          0
rankPoints         0
revives            0
rideDistance       0
roadKills          0
swimDistance       0
teamKills          0
vehicleDestroys    0
walkDistance       0
weaponsAcquired    0
winPoints          0
winPlacePerc       0
dtype: int64

## Simple Test

In [ ]:
n = len(train) * 0.1 
df_sample = sampling(train, n) 
df_sample = get_Output(train)

In [ ]:
features = df_sample.drop(['winPlacePerc'], axis=1)
target = df_sample['winPlacePerc']
LGBM_training_mae(target, features)

In [ ]:
df = get_Output(train)

In [52]:
len(df.columns)

141

In [41]:
# outlier 제거전
features = df.drop(['winPlacePerc'], axis=1)
target = df['winPlacePerc']
LGBM_training_mae(target, features)

0.0599

In [54]:
# outlier 제거후 
LGBM_training_mae(target, features)

0.0602

## Optuna 

In [55]:
X = features
y = target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0xC0FFEE)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=0xC0FFEE)

print(X_train.shape, X_val.shape, X_test.shape, y_train.shape, y_val.shape, y_test.shape)
gc.collect()

(2633724, 140) (877908, 140) (877909, 140) (2633724,) (877908,) (877909,)


43

In [ ]:
K = 5
opt_func = partial(optimizer_LGBM, X=X_train, y=y_train, K=K)

lgbm_study = optuna.create_study(study_name="LGBM", direction="maximize")
lgbm_study.optimize(opt_func, n_trials=5)

[I 2022-06-16 12:52:16,307] A new study created in memory with name: LGBM


[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.244767 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[1]	valid_0's l2: 0.0936925
Training until validation scores don't improve for 25 rounds
[2]	valid_0's l2: 0.0936925
[3]	valid_0's l2: 0.0936925
[4]	valid_0's l2: 0.0936925
[5]	valid_0's l2: 0.0936924
[6]	valid_0's l2: 0.0936924
[7]	valid_0's l2: 0.0936924
[8]	valid_0's l2: 0.0936924
[9]	valid_0's l2: 0.0936923
[10]	valid_0's l2: 0.0936923
[11]	valid_0's l2: 0.0936923
[12]	valid

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.242661 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[1]	valid_0's l2: 0.0936951
Training until validation scores don't improve for 25 rounds
[2]	valid_0's l2: 0.0936951
[3]	valid_0's l2: 0.093695
[4]	valid_0's l2: 0.093695
[5]	valid_0's l2: 0.093695
[6]	valid_0's l2: 0.0936949
[7]	valid_0's l2: 0.0936949
[8]	valid_0's l2: 0.0936949
[9]	valid_0's l2: 0.0936949
[10]	valid_0's l2: 0.0936948
[11]	valid_0's l2: 0.0936948
[12]	valid_0's l2: 0.0936948
[13]	valid_0's l2: 0.0936948
[14]	valid_0's l2: 0.0936947
[15]	valid_0's l2: 0.0936947
[16]	valid_0's l2: 0.093694

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.245827 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[1]	valid_0's l2: 0.0937933
Training until validation scores don't improve for 25 rounds
[2]	valid_0's l2: 0.0937933
[3]	valid_0's l2: 0.0937932
[4]	valid_0's l2: 0.0937932
[5]	valid_0's l2: 0.0937932
[6]	valid_0's l2: 0.0937932
[7]	valid_0's l2: 0.0937931
[8]	valid_0's l2: 0.0937931
[9]	valid_0's l2: 0.0937931
[10]	valid_0's l2: 0.093793
[11]	valid_0's l2: 0.093793
[12]	valid_0's l2: 0.093793
[13]	valid_0's l2: 0.093793
[14]	valid_0's l2: 0.0937929
[15]	valid_0's l2: 0.0937929
[16]	valid_0's l2: 0.0937929
[17]	valid_0's l2: 0.0937929
[18]	valid_0's l2: 0.0937928
[19]	valid_0's l2: 0.0937928
[20]	valid_0's l2: 0.0937928
[21]	valid_0'

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[1]	valid_0's l2: 0.0934788
Training until validation scores don't improve for 25 rounds
[2]	valid_0's l2: 0.0934787
[3]	valid_0's l2: 0.0934787
[4]	valid_0's l2: 0.0934787
[5]	valid_0's l2: 0.0934787
[6]	valid_0's l2: 0.0934786
[7]	valid_0's l2: 0.0934786
[8]	valid_0's l2: 0.0934786
[9]	valid_0's l2: 0.0934785
[10]	valid_0's l2: 0.0934785
[11]	valid_0's l2: 0.0934785
[12]	valid_0's l2: 0.0934785
[13]	valid_0's l2: 0.0934784
[14]	valid_0's l2: 0.0934784
[15]	valid_0's l2: 0.0934784
[16]	valid_0's l2: 0.0934784
[17]	valid_0's l2: 0.0934783
[18]	valid_0's l2: 0.0934783
[19]	valid_0's l2: 0.0934783
[20]	valid_0's l2: 0.0934782
[21]	valid_0's l2: 0.0934782
[22]	valid_0's l2: 0.0934782
[23]	valid_0's l2: 0.0934782
[24]	valid_0's l2: 0.0934781
[25]	valid_0's l2: 0.0934781
[26]	valid_0's l2: 0.0934781
[27]	valid_0's l2: 0.0934781
[28]	valid_0's l2: 0.093478
[29]	va

...

[256]	valid_0's l2: 0.0936921
[257]	valid_0's l2: 0.0936921
[258]	valid_0's l2: 0.0936921
[259]	valid_0's l2: 0.0936921
[260]	valid_0's l2: 0.0936921
[261]	valid_0's l2: 0.0936921
[262]	valid_0's l2: 0.0936921
[263]	valid_0's l2: 0.0936921
[264]	valid_0's l2: 0.0936921
[265]	valid_0's l2: 0.0936921
[266]	valid_0's l2: 0.0936921
[267]	valid_0's l2: 0.0936921
[268]	valid_0's l2: 0.0936921
[269]	valid_0's l2: 0.0936921
[270]	valid_0's l2: 0.0936921
[271]	valid_0's l2: 0.0936921
[272]	valid_0's l2: 0.0936921
[273]	valid_0's l2: 0.0936921
[274]	valid_0's l2: 0.0936921
[275]	valid_0's l2: 0.0936921
[276]	valid_0's l2: 0.0936921
[277]	valid_0's l2: 0.0936921
[278]	valid_0's l2: 0.0936921
[279]	valid_0's l2: 0.0936921
[280]	valid_0's l2: 0.0936921
[281]	valid_0's l2: 0.0936921
[282]	valid_0's l2: 0.0936921
[283]	valid_0's l2: 0.0936921
[284]	valid_0's l2: 0.0936921
[285]	valid_0's l2: 0.0936921
[286]	valid_0's l2: 0.0936921
[287]	valid_0's l2: 0.0936921
[288]	valid_0's l2: 0.0936921
[289]	vali

[531]	valid_0's l2: 0.0936916
[532]	valid_0's l2: 0.0936916
[533]	valid_0's l2: 0.0936916
[534]	valid_0's l2: 0.0936916
[535]	valid_0's l2: 0.0936916
[536]	valid_0's l2: 0.0936916
[537]	valid_0's l2: 0.0936916
[538]	valid_0's l2: 0.0936916
[539]	valid_0's l2: 0.0936916
[540]	valid_0's l2: 0.0936916
[541]	valid_0's l2: 0.0936916
[542]	valid_0's l2: 0.0936916
[543]	valid_0's l2: 0.0936916
[544]	valid_0's l2: 0.0936916
[545]	valid_0's l2: 0.0936916
[546]	valid_0's l2: 0.0936916
[547]	valid_0's l2: 0.0936916
[548]	valid_0's l2: 0.0936916
[549]	valid_0's l2: 0.0936916
[550]	valid_0's l2: 0.0936916
[551]	valid_0's l2: 0.0936916
[552]	valid_0's l2: 0.0936916
[553]	valid_0's l2: 0.0936916
[554]	valid_0's l2: 0.0936916
[555]	valid_0's l2: 0.0936916
[556]	valid_0's l2: 0.0936916
[557]	valid_0's l2: 0.0936916
[558]	valid_0's l2: 0.0936916
[559]	valid_0's l2: 0.0936916
[560]	valid_0's l2: 0.0936916
[561]	valid_0's l2: 0.0936916
[562]	valid_0's l2: 0.0936916
[563]	valid_0's l2: 0.0936916
[564]	vali

[806]	valid_0's l2: 0.0936912
[807]	valid_0's l2: 0.0936912
[808]	valid_0's l2: 0.0936911
[809]	valid_0's l2: 0.0936911
[810]	valid_0's l2: 0.0936911
[811]	valid_0's l2: 0.0936911
[812]	valid_0's l2: 0.0936911
[813]	valid_0's l2: 0.0936911
[814]	valid_0's l2: 0.0936911
[815]	valid_0's l2: 0.0936911
[816]	valid_0's l2: 0.0936911
[817]	valid_0's l2: 0.0936911
[818]	valid_0's l2: 0.0936911
[819]	valid_0's l2: 0.0936911
[820]	valid_0's l2: 0.0936911
[821]	valid_0's l2: 0.0936911
[822]	valid_0's l2: 0.0936911
[823]	valid_0's l2: 0.0936911
[824]	valid_0's l2: 0.0936911
[825]	valid_0's l2: 0.0936911
[826]	valid_0's l2: 0.0936911
[827]	valid_0's l2: 0.0936911
[828]	valid_0's l2: 0.0936911
[829]	valid_0's l2: 0.0936911
[830]	valid_0's l2: 0.0936911
[831]	valid_0's l2: 0.0936911
[832]	valid_0's l2: 0.0936911
[833]	valid_0's l2: 0.0936911
[834]	valid_0's l2: 0.0936911
[835]	valid_0's l2: 0.0936911
[836]	valid_0's l2: 0.0936911
[837]	valid_0's l2: 0.0936911
[838]	valid_0's l2: 0.0936911
[839]	vali

[135]	valid_0's l2: 0.0936949
[136]	valid_0's l2: 0.0936949
[137]	valid_0's l2: 0.0936949
[138]	valid_0's l2: 0.0936949
[139]	valid_0's l2: 0.0936949
[140]	valid_0's l2: 0.0936949
[141]	valid_0's l2: 0.0936949
[142]	valid_0's l2: 0.0936949
[143]	valid_0's l2: 0.0936949
[144]	valid_0's l2: 0.0936949
[145]	valid_0's l2: 0.0936949
[146]	valid_0's l2: 0.0936949
[147]	valid_0's l2: 0.0936948
[148]	valid_0's l2: 0.0936948
[149]	valid_0's l2: 0.0936948
[150]	valid_0's l2: 0.0936948
[151]	valid_0's l2: 0.0936948
[152]	valid_0's l2: 0.0936948
[153]	valid_0's l2: 0.0936948
[154]	valid_0's l2: 0.0936948
[155]	valid_0's l2: 0.0936948
[156]	valid_0's l2: 0.0936948
[157]	valid_0's l2: 0.0936948
[158]	valid_0's l2: 0.0936948
[159]	valid_0's l2: 0.0936948
[160]	valid_0's l2: 0.0936948
[161]	valid_0's l2: 0.0936948
[162]	valid_0's l2: 0.0936948
[163]	valid_0's l2: 0.0936948
[164]	valid_0's l2: 0.0936948
[165]	valid_0's l2: 0.0936948
[166]	valid_0's l2: 0.0936948
[167]	valid_0's l2: 0.0936948
[168]	vali

[410]	valid_0's l2: 0.0936944
[411]	valid_0's l2: 0.0936944
[412]	valid_0's l2: 0.0936944
[413]	valid_0's l2: 0.0936944
[414]	valid_0's l2: 0.0936944
[415]	valid_0's l2: 0.0936944
[416]	valid_0's l2: 0.0936944
[417]	valid_0's l2: 0.0936944
[418]	valid_0's l2: 0.0936944
[419]	valid_0's l2: 0.0936944
[420]	valid_0's l2: 0.0936944
[421]	valid_0's l2: 0.0936944
[422]	valid_0's l2: 0.0936944
[423]	valid_0's l2: 0.0936944
[424]	valid_0's l2: 0.0936944
[425]	valid_0's l2: 0.0936944
[426]	valid_0's l2: 0.0936944
[427]	valid_0's l2: 0.0936944
[428]	valid_0's l2: 0.0936944
[429]	valid_0's l2: 0.0936944
[430]	valid_0's l2: 0.0936944
[431]	valid_0's l2: 0.0936943
[432]	valid_0's l2: 0.0936943
[433]	valid_0's l2: 0.0936943
[434]	valid_0's l2: 0.0936943
[435]	valid_0's l2: 0.0936943
[436]	valid_0's l2: 0.0936943
[437]	valid_0's l2: 0.0936943
[438]	valid_0's l2: 0.0936943
[439]	valid_0's l2: 0.0936943
[440]	valid_0's l2: 0.0936943
[441]	valid_0's l2: 0.0936943
[442]	valid_0's l2: 0.0936943
[443]	vali

In [ ]:
# optuna가 시도했던 모든 실험 관련 데이터
lgbm_study.trials_dataframe()

In [ ]:
print("Best Score: %.4f" % lgbm_study.best_value) # best score 출력
print("Best params: ", lgbm_study.best_trial.params) # best score일 때의 하이퍼파라미터들

In [ ]:
# 실험 기록 시각화
optuna.visualization.plot_optimization_history(lgbm_study)

In [ ]:
# hyper-parameter들의 중요도
optuna.visualization.plot_param_importances(lgbm_study)

In [ ]:
trial = lgbm_study.best_trial
trial_params = trial.params

final_lgb_model = LGBMRegressor(**trial_params)
final_lgb_model.fit(X, y) # finalize model